In [206]:
from transformers import TapasTokenizer, TapasForQuestionAnswering

# Load the model and tokenizer from the local directory
tokenizer = TapasTokenizer.from_pretrained("./Tapas")
model = TapasForQuestionAnswering.from_pretrained("./Tapas")


Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at ./Tapas and are newly initialized: ['output_bias', 'column_output_bias', 'output_weights', 'column_output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [207]:
# Ask Questions
inputs = tokenizer(
    table=order_data,
    queries=["What segment has the highest sales?"],
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)
outputs = model(**inputs)

# Process the outputs to get the answer
logits = outputs.logits
predicted_answer = tokenizer.decode(torch.argmax(logits, dim=-1).squeeze(), skip_special_tokens=True)
print(f"Predicted answer: {predicted_answer}")


Predicted answer: [ u n u s e d 3 4 ]


In [208]:
import os
os.listdir()

['.ipynb_checkpoints',
 'BERT',
 'BERT.ipynb',
 'fine-tuned-tapas',
 'fine_tuned_tabert',
 'fraudTest.csv',
 'fraudTrain.csv',
 'Fraudulent transactions-Bkp02Sept2024.ipynb',
 'Fraudulent transactions-Copy1.ipynb',
 'Fraudulent transactions.ipynb',
 'fraud_detection_nn.pth',
 'fraud_detection_nn_es.pth',
 'LIME_implementation',
 'LIME_implementation.zip',
 'logs',
 'orders.csv',
 'orders_updated.csv',
 'proj2',
 'results',
 'Tapas',
 'Tapas.ipynb',
 'tokenized_data.pt',
 'train_df.csv',
 'train_df_new.csv',
 'Untitled.ipynb']

In [209]:
import chardet

# Detect the encoding of the file
with open("orders.csv", "rb") as f:
    result = chardet.detect(f.read())

# Read the CSV file with the detected encoding
order_data = pd.read_csv("orders.csv", encoding=result['encoding'])

# Display the first few rows of the DataFrame to verify
print(order_data.head())

   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2018-152156   11/8/2018  11/11/2018    Second Class    CG-12520   
1       2  CA-2018-152156   11/8/2018  11/11/2018    Second Class    CG-12520   
2       3  CA-2018-138688   6/12/2018   6/16/2018    Second Class    DV-13045   
3       4  US-2017-108966  10/11/2017  10/18/2017  Standard Class    SO-20335   
4       5  US-2017-108966  10/11/2017  10/18/2017  Standard Class    SO-20335   

     Customer Name    Segment Country/Region             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category  \
0     42420.0   Sout

In [210]:
import matplotlib.pyplot as plt


# Check for missing values
missing_values = order_data.isnull().sum()
print("Missing values in each column:\n", missing_values)

# Check for duplicate rows
duplicate_rows = order_data.duplicated().sum()
print("Number of duplicate rows:", duplicate_rows)

# Optionally, remove duplicate rows
order_data = order_data.drop_duplicates()

# Check data types
data_types = order_data.dtypes
print("Data types of each column:\n", data_types)

# Generate basic statistics
statistics = order_data.describe()
print("Basic statistics of the dataset:\n", statistics)

current_length = len(order_data)
print(f"Current length of the dataset: {current_length}")

order_data=order_data.dropna()

Missing values in each column:
 Row ID             0
Order ID           0
Order Date         0
Ship Date          0
Ship Mode          0
Customer ID        0
Customer Name      0
Segment            0
Country/Region     0
City               0
State              0
Postal Code       11
Region             0
Product ID         0
Category           0
Sub-Category       0
Product Name       0
Sales              0
Quantity           0
Discount           0
Profit             0
dtype: int64
Number of duplicate rows: 0
Data types of each column:
 Row ID              int64
Order ID           object
Order Date         object
Ship Date          object
Ship Mode          object
Customer ID        object
Customer Name      object
Segment            object
Country/Region     object
City               object
State              object
Postal Code       float64
Region             object
Product ID         object
Category           object
Sub-Category       object
Product Name       object
Sales           

In [211]:
current_length = len(order_data)
print(f"Current length of the dataset: {current_length}")

Current length of the dataset: 9983


In [212]:
order_data=order_data[['Segment','Category','Sub-Category']]

In [186]:
order_data

,Segment,Category,Sub-Category
0,Consumer,Furniture,Bookcases
1,Consumer,Furniture,Chairs
2,Corporate,Office Supplies,Labels
3,Consumer,Furniture,Tables
4,Consumer,Office Supplies,Storage
...,...,...,...
9989,Consumer,Furniture,Furnishings
9990,Consumer,Furniture,Furnishings
9991,Consumer,Technology,Phones
9992,Consumer,Office Supplies,Paper


In [152]:
import pandas as pd
from transformers import TapasTokenizer

# Example table data
data = {
    'Segment': ['Consumer', 'Corporate', 'Home Office'],
    'Sales': ['1000', '1500', '500']
}
table = pd.DataFrame(data)

# Example question and answer
questions = ["What segment has the highest sales?"]
answer_coordinates = [[(1, 1)]]  # Coordinates for the answer cell
answer_texts = ["Corporate"]

# Initialize the tokenizer
tokenizer = TapasTokenizer.from_pretrained("./Tapas")

# Tokenize the inputs
inputs = tokenizer(
    table=table,
    queries=questions,
    answer_coordinates=answer_coordinates,
    answer_text=answer_texts,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

print(inputs)


{'input_ids': tensor([[  101,  2054,  6903,  2038,  1996,  3284,  4341,  1029,   102,  6903,
          4341,  7325,  6694,  5971, 10347,  2188,  2436,  3156,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [165]:
import pandas as pd
from transformers import TapasTokenizer

# Example table data
data = {
    'Segment': ['Consumer', 'Corporate', 'Home Office'],
    'Sales': ["1000", "1500", "500"]
}
table = pd.DataFrame(data)

# Example question and answer
questions = ["What segment has the highest sales?"]
answer_coordinates = [[(1, 1)]]  # Coordinates for the answer cell
answer_texts = ["1500"]  # The text in the answer cell

# Initialize the tokenizer
tokenizer = TapasTokenizer.from_pretrained("./Tapas")

# Tokenize the inputs
try:
    inputs = tokenizer(
        table=table,
        queries=questions,
        answer_coordinates=answer_coordinates,
        answer_text=answer_texts,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    print(inputs)
except ValueError as e:
    print(f"An error occurred during tokenization: {e}")


{'input_ids': tensor([[  101,  2054,  6903,  2038,  1996,  3284,  4341,  1029,   102,  6903,
          4341,  7325,  6694,  5971, 10347,  2188,  2436,  3156,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [ ]:
import torch
from transformers import TapasTokenizer, TapasForQuestionAnswering
import pandas as pd


order_data=order_data.head(500)
# Tokenize the entire dataset
tokenizer = TapasTokenizer.from_pretrained("./Tapas")
questions = ["What segment has the highest sales?"] * len(order_data)
answer_coordinates = [[(0, 0)]] * len(order_data)  # Adjust coordinates as needed
answer_texts = ['Consumer'] * len(order_data)  # Adjust answers as needed

inputs = tokenizer(
    table=order_data,
    queries=questions,
    answer_coordinates=answer_coordinates,
    answer_text=answer_texts,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

# Save the tokenized data
torch.save(inputs, "tokenized_data.pt")

# Custom Dataset class to load pre-tokenized data
class CustomDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return self.tokenized_data['input_ids'].shape[0]

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.tokenized_data.items()}

# Load the tokenized data
tokenized_data = torch.load("tokenized_data.pt")

# Create dataset instances
train_dataset = CustomDataset(tokenized_data)
eval_dataset = CustomDataset(tokenized_data)

# Print dataset sizes to verify
print(f"Training dataset size: {len(train_dataset)}")
print(f"Evaluation dataset size: {len(eval_dataset)}")

# Load the model
model = TapasForQuestionAnswering.from_pretrained("./Tapas")

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine-tuned-tapas")
tokenizer.save_pretrained("./fine-tuned-tapas")


In [ ]:
tokenizer = TapasTokenizer.from_pretrained("./fine-tuned-tapas")
model = TapasForQuestionAnswering.from_pretrained("./fine-tuned-tapas")

# Ask Questions
inputs = tokenizer(
    table=order_data,
    queries=["What segment has the highest sales?"],
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)
outputs = model(**inputs)

# Process the outputs to get the answer
logits = outputs.logits
predicted_answer = tokenizer.decode(torch.argmax(logits, dim=-1).squeeze(), skip_special_tokens=True)
print(f"Predicted answer: {predicted_answer}")


In [199]:
# Get the logits directly
logits = outputs.logits

# Assuming logits is a 2D tensor with shape [batch_size, sequence_length]
# Get the predicted token indices for start and end positions
start_logits, end_logits = logits.split(256, dim=1)  # Adjust the split size based on your model's output

# Squeeze the last dimension
start_logits = start_logits.squeeze(-1)
end_logits = end_logits.squeeze(-1)

# Get the predicted start and end positions
start_index = torch.argmax(start_logits, dim=1).item()
end_index = torch.argmax(end_logits, dim=1).item()

# Decode the answer from the table
predicted_answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(inputs['input_ids'][0, start_index:end_index + 1])
)
print(f"Predicted answer: {predicted_answer}   start  {start_index}   end {end_index}")


Predicted answer:    start  14   end 6


In [200]:
inputs['input_ids']

tensor([[  101,  2054,  6903,  2038,  1996,  3284,  4341,  1029,   102,  6903,
          4696,  4942,  1011,  4696,  7325,  7390,  2338, 18382,  2015,  7325,
          7390,  8397,  5971,  2436,  6067, 10873,  7325,  7390,  7251,  7325,
          2436,  6067,  5527,  7325,  7390, 23127,  7325,  2436,  6067,  2396,
          7325,  2974, 11640,  7325,  2436,  6067, 14187,  2545,  7325,  2436,
          6067, 22449,  7325,  7390,  7251,  7325,  2974, 11640,  7325,  2436,
          6067,  3259,  7325,  2436,  6067, 14187,  2545,  2188,  2436,  2436,
          6067, 22449,  2188,  2436,  2436,  6067, 14187,  2545,  7325,  2436,
          6067,  5527,  7325,  2436,  6067,  5527,  7325,  2436,  6067,  2396,
          7325,  2974, 11640,  7325,  2436,  6067, 14187,  2545,  5971,  2436,
          6067,  2396,  5971,  2436,  6067, 22449,  7325,  7390,  8397,  7325,
          7390,  7251,  7325,  2436,  6067, 14187,  2545,  7325,  2974, 16611,
          7325,  7390,  2338, 18382,  2015,  7325,  

In [192]:
print(logits.shape)


torch.Size([1, 512])


In [166]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import TapasTokenizer, TapasForQuestionAnswering, Trainer, TrainingArguments
import pandas as pd

# Ensure order_data is defined and loaded
order_data = pd.DataFrame({
    'Segment': ['Consumer', 'Corporate', 'Home Office'],
    'Sales': [1000, 1500, 500]
})

current_length = len(order_data)
rows_to_drop = current_length % 8
if rows_to_drop > 0:
    order_data = order_data[:-rows_to_drop]

# Reset the index and convert all values to strings
order_data.reset_index(drop=True, inplace=True)
order_data = order_data.astype(str)

# Save the updated dataset to a new CSV file
order_data.to_csv("orders_updated.csv", index=False)

# Generate questions and answers for each row
questions = ["What segment has the highest sales?"] * len(order_data)
answer_coordinates = [[(0, 0)]] * len(order_data)  # Adjust coordinates as needed
answer_texts = ["Consumer"] * len(order_data)  # Adjust answers as needed

assert len(questions) == len(order_data), "Questions list length does not match order_data length"
assert len(answer_coordinates) == len(order_data), "Answer coordinates list length does not match order_data length"
assert len(answer_texts) == len(order_data), "Answer texts list length does not match order_data length"

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, questions, answer_coordinates, answer_texts):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.questions = questions
        self.answer_coordinates = answer_coordinates
        self.answer_texts = answer_texts

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if idx >= len(self.dataframe):
            raise IndexError(f"Index {idx} is out of bounds for dataframe with length {len(self.dataframe)}")

        row = self.dataframe.iloc[idx]
        table = pd.DataFrame([row])
        print(f"Processing row {idx}: {row}")  # Debugging statement

        try:
            inputs = self.tokenizer(
                table=table,
                queries=[self.questions[idx]],
                answer_coordinates=[self.answer_coordinates[idx]],
                answer_text=[self.answer_texts[idx]],
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            )
        except Exception as e:
            print(f"Error during tokenization at index {idx}: {e}")
            raise e

        return inputs

# Create instances of the custom dataset
tokenizer = TapasTokenizer.from_pretrained("./Tapas")
train_dataset = CustomDataset(order_data, tokenizer, questions, answer_coordinates, answer_texts)
eval_dataset = CustomDataset(order_data, tokenizer, questions, answer_coordinates, answer_texts)

# Print dataset sizes to verify
print(f"Training dataset size: {len(train_dataset)}")
print(f"Evaluation dataset size: {len(eval_dataset)}")

# Load the model from the local directory
model = TapasForQuestionAnswering.from_pretrained("./Tapas")

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Initialize the Trainer with the custom DataLoader
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine-tuned-tapas")
tokenizer.save_pretrained("./fine-tuned-tapas")

# Load the fine-tuned model
tokenizer = TapasTokenizer.from_pretrained("./fine-tuned-tapas")
model = TapasForQuestionAnswering.from_pretrained("./fine-tuned-tapas")

# Ask Questions
inputs = tokenizer(
    table=order_data,
    queries=["What segment has the highest sales?"],
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)
outputs = model(**inputs)

# Process the outputs to get the answer
logits = outputs.logits
predicted_answer = tokenizer.decode(torch.argmax(logits, dim=-1).squeeze(), skip_special_tokens=True)
print(f"Predicted answer: {predicted_answer}")


Training dataset size: 0
Evaluation dataset size: 0


Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at ./Tapas and are newly initialized: ['output_bias', 'column_output_bias', 'output_weights', 'column_output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [173]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import TapasTokenizer, TapasForQuestionAnswering, Trainer, TrainingArguments
import pandas as pd

current_length = len(order_data)
rows_to_drop = current_length % 8
if rows_to_drop > 0:
    order_data = order_data[:-rows_to_drop]

# Reset the index and convert all values to strings
order_data.reset_index(drop=True, inplace=True)
order_data = order_data.astype(str)

# Save the updated dataset to a new CSV file
order_data.to_csv("orders_updated.csv", index=False)

# Generate questions and answers for each row
questions = ["What segment has the highest sales?"] 
answer_coordinates = [[(0, 0)]]   # Adjust coordinates as needed
answer_texts = ['Consumer']  # Adjust answers as needed
#assert len(questions) == len(order_data), "Questions list length does not match order_data length"
#assert len(answer_coordinates) == len(order_data), "Answer coordinates list length does not match order_data length"
#assert len(answer_texts) == len(order_data), "Answer texts list length does not match order_data length"
# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, questions, answer_coordinates, answer_texts):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.questions = questions
        self.answer_coordinates = answer_coordinates
        self.answer_texts = answer_texts

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if idx >= len(self.dataframe):
            raise IndexError(f"Index {idx} is out of bounds for dataframe with length {len(self.dataframe)}")

        row = self.dataframe.iloc[idx]
        table = pd.DataFrame([row])
        print(f"Processing row {idx}: {row}")  # Debugging statement

        try:
            
            inputs = self.tokenizer(
                table=table,
                queries=questions,
                answer_coordinates=answer_coordinates,
                answer_text=answer_texts,
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            )
        except Exception as e:
            print(f"Error during tokenization at index {idx}: {e}")
            raise e

        return inputs, idx

# Create instances of the custom dataset
tokenizer = TapasTokenizer.from_pretrained("./Tapas")
train_dataset = CustomDataset(order_data, tokenizer, questions, answer_coordinates, answer_texts)
eval_dataset = CustomDataset(order_data, tokenizer, questions, answer_coordinates, answer_texts)

# Print dataset sizes to verify
print(f"Training dataset size: {len(train_dataset)}")
print(f"Evaluation dataset size: {len(eval_dataset)}")

# Load the model from the local directory
model = TapasForQuestionAnswering.from_pretrained("./Tapas")

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Initialize the Trainer with the custom DataLoader
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine-tuned-tapas")
tokenizer.save_pretrained("./fine-tuned-tapas")

# Load the fine-tuned model
tokenizer = TapasTokenizer.from_pretrained("./fine-tuned-tapas")
model = TapasForQuestionAnswering.from_pretrained("./fine-tuned-tapas")

# Ask Questions
inputs = tokenizer(table=order_data, queries=["What segment has the highest sales?"], padding="max_length", return_tensors="pt")
outputs = model(**inputs)
print(outputs)

Training dataset size: 9976
Evaluation dataset size: 9976


Some weights of TapasForQuestionAnswering were not initialized from the model checkpoint at ./Tapas and are newly initialized: ['output_bias', 'column_output_bias', 'output_weights', 'column_output_weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Anaconda3\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Processing row 7400: Segment                Consumer
Category        Office Supplies
Sub-Category           Supplies
Name: 7400, dtype: object
Error during tokenization at index 7400: iloc cannot enlarge its target object


IndexError: iloc cannot enlarge its target object

In [141]:
answer_texts[1]

'Consumer'

In [145]:
order_data.iloc[7400]

Segment                Consumer
Category        Office Supplies
Sub-Category           Supplies
Name: 7400, dtype: object

In [146]:
train_dataset[1]

Processing row 1: Segment          Consumer
Category        Furniture
Sub-Category       Chairs
Name: 1, dtype: object
Error during tokenization at index 1: iloc cannot enlarge its target object


IndexError: iloc cannot enlarge its target object

In [ ]:

# Step 5: Load the Fine-Tuned Model
tokenizer = TapasTokenizer.from_pretrained("./fine-tuned-tapas")
model = TapasForQuestionAnswering.from_pretrained("./fine-tuned-tapas")

# Step 6: Ask Questions
# Example question: Is the premium paid higher for males?
inputs = tokenizer(table=table, queries=["what segament has highest sales?"], padding="max_length", return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
predicted_answer_coordinates = tokenizer.convert_logits_to_predictions(inputs, logits)
print(predicted_answer_coordinates)

# Example question: What is the average premium paid by gender?
inputs = tokenizer(table=table, queries=["What is the average sales by category?"], padding="max_length", return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
predicted_answer_coordinates = tokenizer.convert_logits_to_predictions(inputs, logits)
print(predicted_answer_coordinates)

In [18]:
print(f"Training dataset size: {len(train_dataset)}")
print(f"Evaluation dataset size: {len(eval_dataset)}")



Training dataset size: 7995
Evaluation dataset size: 1999
